In [5]:

library(dplyr)
library(vegan)

ERROR: Error in library(arrow): there is no package called ‘arrow’


In [23]:
df <- read.csv('../data/Associations/moms-pi/rkbm_abnd_mat/graph_02.abnd_mat_filtered.csv', row.names=1) %>%
    t()
outcome = read.csv('../data/Associations/moms-pi/outcome.csv', row.names=1)
outcome

,outcome
,<int>
SRR6744869,0
SRR6744974,0
SRR6744905,0
SRR6744556,0
SRR6745162,0
SRR6743958,0
SRR6743926,1
SRR6745123,0
SRR6744106,0


In [89]:
df_filt = df
df_filt[df_filt < 50] <- 0
non_zero_prop = colSums(df_filt != 0)/nrow(df_filt)
df_filt = df_filt[, non_zero_prop > 0]
print(dim(df_filt))
print(dim(df))


[1]     87 153079
[1]     87 515453


In [127]:
group = factor(outcome$outcome)
adonis_result <- adonis2( df_filt ~ group, method = 'bray', permutations = 999)
adonis_result = as.data.frame(adonis_result)
reslist = list()
for (i in 1:5) {
    reslist[[length(reslist)+1]] <- adonis_result
}
res_final= do.call(rbind, reslist)
res_final = cbind(res_final, dof = sub("\\d+$", "", row.names(res_final)))
rownames(res_final) = seq_len(nrow(res_final))
res_final

,Df,SumOfSqs,R2,F,Pr(>F),dof
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,1,0.7431904,0.02209007,1.920071,0.021,Model
2,85,32.9004453,0.97790993,NA,NA,Residual
3,86,33.6436357,1.00000000,NA,NA,Total
4,1,0.7431904,0.02209007,1.920071,0.021,Model
5,85,32.9004453,0.97790993,NA,NA,Residual
6,86,33.6436357,1.00000000,NA,NA,Total
7,1,0.7431904,0.02209007,1.920071,0.021,Model
8,85,32.9004453,0.97790993,NA,NA,Residual
9,86,33.6436357,1.00000000,NA,NA,Total


In [132]:
props = c(0, 0.1)
rkbm_ts = c(10, 50)
methods = c('euclidean', 'bray')
get_adonis <- function(X, group) {
  res_list = list()
  for (p in props) {
    for (t in rkbm_ts) {
      for (m in methods) {
        X_filt = X
        X_filt[X_filt < t] <- 0
        non_zero_prop = colSums(X_filt != 0)/nrow(X_filt)
        X_filt = X_filt[, non_zero_prop > p]
        print(dim(X))
        print(dim(X_filt))
        res = adonis2(X_filt ~ group, method=m)
        res = cbind(res, rbkm_t = t, nonz_prop = p, method = m)
        res_list[[length(res_list)+1]] <- res
      }
    }
  }
  res_final = do.call(rbind, res_list)
  res_final = cbind(res_final, dof = sub("\\d+$", "", row.names(res_final)))
  rownames(res_final) = seq_len(nrow(res_final))
  return(res_final)
}
res_final = get_adonis(df, factor(outcome$outcome))
write.csv(res_final, "adonis_result.csv", row.names = FALSE, quote = FALSE)


[1]     87 515453
[1]     87 399630
[1]     87 515453
[1]     87 399630
[1]     87 515453
[1]     87 153079
[1]     87 515453
[1]     87 153079
[1]     87 515453
[1]    87 12991
[1]     87 515453
[1]    87 12991
[1]     87 515453
[1]   87 7801
[1]     87 515453
[1]   87 7801


In [20]:
write.csv(as.data.frame(adonis_result), "adonis_result.csv", row.names = TRUE)

In [64]:
# Example n x p matrix (rows = samples, columns = features)
set.seed(123)  # For reproducibility

# Step 1: Calculate the dissimilarity matrix (e.g., Bray-Curtis)
dissimilarity_matrix <- vegdist(df, method = "euclidean")

# Step 2: Perform NMDS
nmds_result <- metaMDS(dissimilarity_matrix, k = 2, trymax = 100)

# Step 3: View the results
print(nmds_result)

# Step 4: Plot the NMDS
plot(nmds_result, type = "t")


,Df,SumOfSqs,R2,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Model,1,6.519956e+10,0.01308253,1.126756,0.2907
Residual,85,4.918512e+12,0.98691747,NA,NA
Total,86,4.983711e+12,1.00000000,NA,NA
